In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from furl import furl

In [ ]:
reg_df = pd.read_excel('2-13.xlsx')

In [ ]:
reg_df

In [ ]:
reg_df.dtypes

In [ ]:
reg_df['注册时间'] = pd.to_datetime(reg_df['注册时间'])  #设置时间

In [ ]:
reg_df.dtypes

In [ ]:
reg_df['月'] = reg_df['注册时间'].dt.month # 分离出月

In [ ]:
def what_port(row):
    '''
    区分是哪个端来的
    :param row:
    :return:
    '''
    if pd.isna(row['注册来源url']) and pd.isna(row['注册的前一页url']):
        return 'app'
    else:
        return 'PC'

In [ ]:
reg_df['端'] = reg_df.apply(what_port,axis=1)

In [ ]:
port_ana = reg_df[['月','端']].groupby('月').value_counts()

In [ ]:
def no_name(url):
    '''
    判断是否是域来的,如果是，再对url进行解析，不是就返回host
    :param url:
    :return:
    '''
    f_obj = furl(url)
    if f_obj.host == 'db.yaozh.com':
        return f_obj.path.segments[0]
    return f_obj.host

In [ ]:
def get_par(url):
    '''
    解析url,获取来源
    :param url:
    :return:
    '''
    if url is np.NaN:
        return 'app'

    url_params_dict =  furl(url).query.params
    source = url_params_dict.get('ga_name',url_params_dict.get('backurl', url_params_dict.get('back_url', url_params_dict.get('ga_source','/'))))
    if source is not None and "http" in source:
        return no_name(source)
    else:
        return source

In [ ]:
def real_source(row):
    '''
    两个来源的综合判断
    :param row:
    :return:
    '''
    if row['注册来源_first_dir'] != 'app':
        if row['注册来源_first_dir'] == '/' and row['注册的前一页url_first_dir'] == '/':
            return '/'
        elif row['注册来源_first_dir'] != '/' and row['注册的前一页url_first_dir'] == '/':
            return row['注册来源_first_dir']
        elif row['注册来源_first_dir'] == '/' and row['注册的前一页url_first_dir'] != '/':
            return row['注册的前一页url_first_dir']
        else:
            return row['注册来源_first_dir']
    return 'app'

In [ ]:
import numpy as np

In [ ]:
reg_df['注册来源_first_dir'] = reg_df['注册来源url'].apply(get_par)

In [ ]:
reg_df['注册的前一页url_first_dir'] = reg_df['注册的前一页url'].apply(get_par)

In [ ]:
reg_df[['注册来源url','注册来源_first_dir','注册的前一页url','注册的前一页url_first_dir']]

In [ ]:
reg_df['real_source_dir'] = reg_df.apply(real_source, axis=1)